In [ ]:
import numpy as np
from tqdm import tqdm

import xarray
import glob
from pathlib import Path


import sys
sys.path.append('../')
from toffy import streak_detection as sd
from ark.utils import load_utils

In [ ]:
fov_dirs = list(map(Path, glob.glob("../toffy/data/streak/*/")))


streak_data = {}

for fov_dir in tqdm(fov_dirs):
    streak_data[fov_dir.stem] = sd.streak_correction(
        fov_dir, image_name="Noodle", mask_data=True
    )
    streak_data[fov_dir.stem].save_data(all_data=True)
